In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from quantum.Quantum import QuantumRegressor

os.environ["OMP_NUM_THREADS"] = "12"

In [ ]:
# Global variables
OPTIMIZER = None
SHOTS = None
X_DIM = None
BACKEND = None
DEVICE = None
SCALE_FACTORS = None
ANSATZ = None
ENCODER = None
POSTPROCESS = None
ERROR_MITIGATION = None
LAYERS = None
TOKEN = None
HYPERPARAMETERS = None
RE_UPLOAD_DEPTH = None
MAX_ITER = None
TOLERANCE = None
NUM_QUBITS = None
BATCH_SIZE = None
NUM_CORES = None
############################################
# Utility functions
############################################


def parse_settings(settings_file):
    with open(settings_file, 'r') as fp:
        settings = json.load(fp)

    global OPTIMIZER
    OPTIMIZER = settings['OPTIMIZER']

    global SHOTS
    SHOTS = settings['SHOTS']
                
    global BACKEND
    BACKEND = settings['BACKEND']

    global DEVICE
    DEVICE = settings['DEVICE']

    global SCALE_FACTORS
    SCALE_FACTORS = settings['SCALE_FACTORS']

    global POSTPROCESS
    POSTPROCESS = settings['POSTPROCESS']

    global ERROR_MITIGATION
    ERROR_MITIGATION = settings['ERROR_MITIGATION']

    global LAYERS
    LAYERS = settings['LAYERS']

    global HYPERPARAMETERS
    HYPERPARAMETERS = settings['HYPERPARAMETERS']
    # f was removed from HYPERPARAMETERS, this ensures old settings files can still run.
    if 'f' in HYPERPARAMETERS.keys():
        _ = HYPERPARAMETERS.pop('f', None)

    global RE_UPLOAD_DEPTH
    RE_UPLOAD_DEPTH = settings['RE-UPLOAD_DEPTH']

    global MAX_ITER
    MAX_ITER = settings['MAX_ITER']

    global TOLERANCE
    try:
        TOLERANCE = settings['TOLERANCE']
    except KeyError:
        TOLERANCE = None

    global NUM_QUBITS
    try:
        NUM_QUBITS = settings['NUM_QUBITS']
    except KeyError:
        NUM_QUBITS = None

    # classes aren't JSON serializable, so we store the key in the settings file and access it here.
    global ANSATZ
    ANSATZ = ANSATZ_LIST[settings['ANSATZ']]

    global ENCODER
    ENCODER = ENCODER_LIST[settings['ENCODER']]

    global BATCH_SIZE
    BATCH_SIZE = settings['BATCH_SIZE']
    
    global NUM_CORES
    NUM_CORES = settings['NUM_CORES']

def load_dataset(file):
    print(f'Loading dataset from {file}... ')
    data = joblib.load(file)
    X = data['X']
    y = data['y']

    global X_DIM
    _, X_DIM = X.shape
    print(f'Successfully loaded {file} into X and y data. ')
    return X, y


In [ ]:
def create_kwargs():
    #  First have to apply specific ansatz settings: setting number of layers and the number of wires based on features
    ANSATZ.layers = LAYERS
    ANSATZ.set_wires(range(X_DIM))

    kwargs = {
        'encoder': ENCODER,
        'variational': ANSATZ,
        'num_qubits': X_DIM,
        'optimizer': OPTIMIZER,
        # 'optimizer': "BFGS",
        'max_iterations': MAX_ITER,
        'tol': TOLERANCE,
        'device': DEVICE,
        'shots': SHOTS,
        'backend': BACKEND,
        'postprocess': POSTPROCESS,
        'error_mitigation': ERROR_MITIGATION,
        'token': TOKEN,
        're_upload_depth': RE_UPLOAD_DEPTH,
        'batch_size': BATCH_SIZE,
        'njobs':NUM_CORES
    }
    return kwargs

In [ ]:
settings="./M_Full-CRX/M_Full-CRX.json"
# settings="./IQP_Full-Pauli-CRZ/IQP_Full-Pauli-CRZ.json"
train_set="./linear_train.bin"
test_set="./linear_test.bin"
scaler="./linear_scaler.bin"

In [ ]:
X_train, y_train = load_dataset(train_set)
parse_settings(settings)
if DEVICE == 'qiskit.ibmq':
    save_token(instance, token)

global NUM_QUBITS
global X_DIM
if NUM_QUBITS is not None:
    X_DIM = NUM_QUBITS
elif X_DIM == 1:  # if X_DIM is None and num_qubits wasn't specified anywhere use a default value of 2.
    NUM_QUBITS = 2
    X_DIM = NUM_QUBITS

kwargs = create_kwargs()
title=False
if title is None:
    title = os.path.basename(settings)
    title, _ = os.path.splitext(title)



if test_set is not None:
    X_test, y_test = load_dataset(test_set)
else:
    X_test, y_test = None, None

scaler = joblib.load(scaler)

In [ ]:
from qiskit.circuit.library import n_local

class TwoLocal:

    def __init__(self,
                 wires: list = None,
                 entanglement: str = 'linear',
                 reps: int = 1,
                 rot_gates: list = None,
                 entangle_gates: list = None,
                 skip_final_rot: bool = True
                 ):
        self._entanglement = entanglement
        self._reps = reps
        self._rot_gates = rot_gates
        self._entangle_gates = entangle_gates
        self._skip_final_rot = skip_final_rot
        self._wires = None
        self._qc = None
        print(self._wires)
        if wires is not None:
            self.set_wires(wires)
        self._qc.draw('mpl')
        plt.show()
        print(self._wires)
    def __call__(self, parameters, wires: list = None):
        if wires is not None:
            if len(self._wires) != len(self._wires):
                raise ValueError("Cannot override wires instance of different length")
            self._wires = wires
            print(self._wires)
            
        qc = self._qc
        if qc.num_parameters_settable != len(parameters):
            raise ValueError("Incorrect number of parameters. Expected ", qc.num_parameters_settable, "but received ",
                             len(parameters))
        qc = qc.decompose()
        parameters = parameters.tolist()
        qc = qc.assign_parameters(parameters)
        qml_circuit = qml.from_qiskit(qc)
        print(self._wires)
        qml_circuit(wires=self._wires)

    @property
    def num_params(self):
        return self._qc.num_parameters_settable

    @property
    def layers(self):
        return self._reps

    @layers.setter
    def layers(self, val):
        self._reps = val

    def set_wires(self, wires):
        self._wires = wires
        print(self._wires)
        if self._entanglement == 'complete':
            entanglement = []
            for i in wires:
                for j in wires:
                    if i != j:
                        entanglement.append((i, j))
            self._entanglement = entanglement
        self._qc = n_local.TwoLocal(num_qubits=len(self._wires), entanglement=self._entanglement, reps=self._reps,
                                    rotation_blocks=self._rot_gates, entanglement_blocks=self._entangle_gates,
                                    skip_final_rotation_layer=self._skip_final_rot)

In [ ]:
# from quantum.circuits.Ansatz import TwoLocal, EfficientSU2
num_qubits = 5
wires = range(num_qubits)
device = qml.device('qulacs.simulator', wires=num_qubits)
# variational = EfficientSU2(wires=wires,su2_gates=['ry', 'rz'],entanglement='linear',reps=1,skip_final_rot=True)
variational = TwoLocal(wires=wires,rot_gates=['rx', 'rz'], entangle_gates=['crx'], entanglement='complete')
LAYERS=1
re_upload_depth=1

variational.layers = LAYERS
# variational.set_wires(wires)

num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
feat = generator.uniform(-1, 1, num_qubits)

def circuit(parameters, wires):
    variational(parameters=parameters, wires=wires)
    return  qml.expval(qml.PauliZ(0))

qnode = qml.QNode(circuit,device)
# qml.draw_mpl(qnode)(initial_parameters, wires)
qnode(initial_parameters,wires)

In [ ]:
model = QuantumRegressor(**kwargs)
model.fit(X_train, y_train)

In [ ]:
plt.scatter(model.predict(X_train),y_train)
plt.scatter(model.predict(X_test),y_test)
plt.scatter(X_train,y_train)